In [1]:
import pandas as pd
import pyam
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import openpyxl
import aneris

In [2]:
image_path = '../../../data/TOD/model_results/raw/Consolidated_IMAGE_GEO7_R3_results_12022025_0956.xlsx'
mimosa_path = '../../../data/TOD/model_results/raw/MIMOSA_processed_2.xlsx'
gem_e3_path = '../../../data/TOD/model_results/raw/GEM_E3_v2.xlsx'

In [3]:
regions_un = [
    'Latin America and Caribbean (UN-R5)',
    'Western Europe and Other States (UN-R5)',
    'World', 'Africa (UN-R5)',
    'Asia and the Pacific (UN-R5)',
    'Eastern Europe (UN-R5)']

regions_chpt_19 = ['IMAGE 3.3|Canada',
            'IMAGE 3.3|Japan',
            'IMAGE 3.3|Mexico',
            'IMAGE 3.3|South Africa',
            'IMAGE 3.3|Turkey',
            'IMAGE 3.3|India',
            'IMAGE 3.3|Rest of Southern Africa',
            'IMAGE 3.3|Brazil',
            'IMAGE 3.3|China Region',
            'IMAGE 3.3|Eastern Africa',
            'IMAGE 3.3|Indonesia Region',
            'IMAGE 3.3|Northern Africa',
            'IMAGE 3.3|Rest of South Asia',
            'IMAGE 3.3|Russia Region',
            'IMAGE 3.3|USA',
            'IMAGE 3.3|Western Africa',
            'IMAGE 3.3|Western Europe',
            'IMAGE 3.3|Oceania']

df_image_all = pyam.IamDataFrame(image_path)

gdp_image = df_image_all.filter(variable=['GDP*'], year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
# gdp_image_un = gdp_image.filter(region=regions_un, year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
# gdp_image_19 = gdp_image.filter(region=regions_chpt_19, year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
# gdp_image.variable

[INFO] 11:22:30 - pyam.core: Reading file ..\..\..\data\TOD\model_results\raw\Consolidated_IMAGE_GEO7_R3_results_12022025_0956.xlsx


In [4]:
def round_to_0_1_percent(value):
    if value == 0:
        return 0
    # Determine the number of significant digits to round to
    magnitude = np.floor(np.log10(abs(value))) - 2
    rounding_factor = 10 ** magnitude
    return np.round(value / rounding_factor) * rounding_factor

In [5]:
def compute_3_groups_image(df, variables, weight=None):
    df_high_income = df.aggregate_region(variables,
                              region='High-Income',
                              subregions=[
                                'CAN',
                                'JAP',
                                'USA',
                                'WEU',
                                'OCE'
                              ],
                              weight=weight
                              )

    df_middle_income = df.aggregate_region(variables,
                              region='Middle-Income',
                              subregions=[
                                'MEX',
                                'SAF',
                                'TUR',
                                'BRA',
                                'CHN',
                                'INDO',
                                'RUS',
                              ],
                              weight=weight
                              )

    df_low_income = df.aggregate_region(variables,
                              region='Low-Income',
                              subregions=[
                                'INDIA',
                                'RSAF',
                                'EAF',
                                'NAF',
                                'RSAS',
                                'WAF',
                              ],
                              weight=weight
                              )

    df_3_groups = pyam.concat([df_high_income, df_middle_income, df_low_income])
    return df_3_groups

def compute_UN_groups_image(df, variables, weight=None):
    df_africa = df.aggregate_region(variables,
                              region='Africa (UN-R5)',
                              subregions=[
                                    'NAF',
                                    'WAF',
                                    'EAF',
                                    'SAF',
                                    'RSAF',
                              ],
                              weight=weight
                              )

    df_asia = df.aggregate_region(variables,
                              region='Asia and the Pacific (UN-R5)',
                              subregions=[
                                'CHN',
                                'INDIA',
                                'INDO',
                                'JAP',
                                'KOR',
                                'ME',
                                'RSAS',
                                'SEAS',
                                'STAN',
                              ],
                              weight=weight
                              )

    df_ee = df.aggregate_region(variables,
                              region='Eastern Europe (UN-R5)',
                              subregions=[
                                    'CEU',
                                    'RUS',
                                    'UKR',
                              ],
                              weight=weight
                              )

    df_latin = df.aggregate_region(variables,
                              region='Latin America and Caribbean (UN-R5)',
                              subregions=[
                                    'BRA',
                                    'MEX',
                                    'RCAM',
                                    'RSAM',
                              ],
                              weight=weight
                              )

    df_we = df.aggregate_region(variables,
                              region='Western Europe and Other States (UN-R5)',
                              subregions=[
                                        'CAN',
                                        'OCE',
                                        'TUR',
                                        'USA',
                                        'WEU',
                              ],
                              weight=weight
                              )
    df_world = df.filter(variable=variables, region='World')

    df_un_groups = pyam.concat([df_africa, df_asia, df_ee, df_latin, df_we, df_world])
    return df_un_groups

def compute_3_groups_gem(df, variables, weight=None):
    df_high_income = df.aggregate_region(variables,
                              region='High-Income',
                              subregions=[
                                'USA',
                                'CAN',
                                'JPN',
                                'OCE',
                                'EU28'
                              ],
                              weight=weight
                              )

    df_middle_income = df.aggregate_region(variables,
                              region='Middle-Income',
                              subregions=[
                                'MEX',
                                'SAF',
                                'TUR',
                                'BRA',
                                'CHN',
                                'IDN',
                                'RUS',
                              ],
                              weight=weight
                              )

    df_low_income = df.aggregate_region(variables,
                              region='Low-Income',
                              subregions=[
                                'IND',
                                'ROW',
                              ],
                              weight=weight
                              )

    df_3_groups = pyam.concat([df_high_income, df_middle_income, df_low_income])
    return df_3_groups

def compute_UN_groups_gem(df, variables, weight=None):
    
    df_africa = df.aggregate_region(variables,
                              region='Africa (UN-R5)',
                              subregions=[
                                    'SAF', 
                                    'ROW'
                              ],
                              weight=weight
                              )

    df_asia = df.aggregate_region(variables,
                              region='Asia and the Pacific (UN-R5)',
                              subregions=[
                                'JPN', 
                                'CHN',
                                'IND',
                                'KOR',
                                'IDN',
                                'SAR',
                                'REP',
                              ],
                              weight=weight
                              )

    df_ee = df.aggregate_region(variables,
                              region='Eastern Europe (UN-R5)',
                              subregions=[
                                'REU',
                                'RUS',
                              ],
                              weight=weight
                              )

    df_latin = df.aggregate_region(variables,
                              region='Latin America and Caribbean (UN-R5)',
                              subregions=[
                                'ARG',
                                'BRA',
                                'MEX',
                              ],
                              weight=weight
                              )

    df_we = df.aggregate_region(variables,
                              region='Western Europe and Other States (UN-R5)',
                              subregions=[
                                'EU28',
                                'USA',
                                'CAN',
                                'TUR',
                                'OCE',
                              ],
                              weight=weight
                              )
    
    df_world = df.filter(variable=variables, region='World')

    df_un_groups = pyam.concat([df_africa, df_asia, df_ee, df_latin, df_we, df_world])
    return df_un_groups


# Calculate Convertion Ratio

In [6]:

gdp_image_un = compute_UN_groups_image(gdp_image, gdp_image.variable)
df_un_ratio = gdp_image_un.divide('GDP|PPP', 'GDP|MER', 'GDP|Ratio', ignore_units='')
df_un_ppp_input = gdp_image_un.multiply('GDP|PPP', 1, 'GDP|PPP|Input', ignore_units='billion USD_2010/yr')
df_un_image = pyam.concat([df_un_ppp_input, df_un_ratio])

df_3_groups_image = compute_3_groups_image(gdp_image, gdp_image.variable)
df_3_ratio = df_3_groups_image.divide('GDP|PPP', 'GDP|MER', 'GDP|Ratio', ignore_units='')
df_3_ppp_input = df_3_groups_image.multiply('GDP|PPP', 1, 'GDP|PPP|Input', ignore_units='billion USD_2010/yr')
df_3_image = pyam.concat([df_3_ppp_input, df_3_ratio])
df_3_image.region

['High-Income', 'Low-Income', 'Middle-Income']

# MIMOSA and GEM-E3 Income groups

In [7]:
df_unclean_mimosa = pd.read_excel('../../../data/TOD/model_results/raw/MIMOSA_processed_2.xlsx')
ratio_mask = df_unclean_mimosa['Unit'] == 'fraction of GDP'
# Apply rounding to the 'value' column for rows where the 'unit' column is '%'
# df_unclean_mimosa.loc[ratio_mask, df_unclean_mimosa.columns[5:]] = df_unclean_mimosa.loc[ratio_mask, df_unclean_mimosa.columns[5:]]  * 100
# df_unclean_mimosa.loc[ratio_mask, 'Unit'] = "% of GDP"

df_mid_clean_mimosa = pyam.IamDataFrame(df_unclean_mimosa)
df_mid_clean_mimosa.rename(scenario={
        "REF": "REF-v2",
        "LIFE-TP": "LIFE-TP-v2",
        "TECH-TP": "TECH-TP-v2",
    }, inplace=True)


df_un_mimosa = compute_UN_groups_image(df_mid_clean_mimosa, 'GDP|Climate Damage Corrected|Loss|Relative', weight='GDP|Climate Damage Corrected')
df_3_goups_mimosa = compute_3_groups_image(df_mid_clean_mimosa, 'GDP|Climate Damage Corrected|Loss|Relative', weight='GDP|Climate Damage Corrected')
df_3_goups_mimosa.timeseries()

2020  \
model  scenario   region        variable                                   unit                    
MIMOSA LIFE-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
       REF-v2     High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
       TECH-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   

                                                                                                2030  \
model  scenario   region        variable                                   unit                        
MIMOSA LIFE-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.003938   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.015010   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.006037   
       REF-v2     High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.004683   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.018208   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.006948   
       TECH-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.003921   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.014938   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.006016   

                                                                                                2040  \
model  scenario   region        variable                                   unit                        
MIMOSA LIFE-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.007391   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.026827   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.011207   
       REF-v2     High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.011475   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.043599   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.015606   
       TECH-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.007393   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.026844   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.011210   

                                                                                                2050  \
model  scenario   region        variable                                   unit                        
MIMOSA LIFE-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.008042   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.026514   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.013275   
       REF-v2     High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.020172   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fr

In [8]:
df_unclean_gem = pd.read_excel('../../../data/TOD/model_results/raw/GEM_E3_v2.xlsx')
df_unclean_gem['Scenario'] = df_unclean_gem['Scenario'].replace({'DECARB_LIFE': 'LIFE-TP-v2',
                                                        'DECARB_TECH': 'TECH-TP-v2',
                                                        'REF_V2':'REF-v2',
                                                  })

df_mid_clean_gem = pyam.IamDataFrame(df_unclean_gem)
df_mid_clean_gem.filter(variable=["GDP|MER", 'Population'])
df_mid_clean_gem.rename(unit={'billion US$2010/yr':'billion USD_2010/yr'}, inplace=True)

df_un_gem = compute_UN_groups_gem(df_mid_clean_gem, ["GDP|MER", 'Population'])
df_3_groups_gem = compute_3_groups_gem(df_mid_clean_gem, ["GDP|MER", 'Population'])
df_3_groups_gem.timeseries()

2015  \
model        scenario   region        variable   unit                                
GEM-E3_V2023 LIFE-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  41527.174647   
                                      Population million               1032.375424   
                        Low-Income    GDP|MER    billion USD_2010/yr   8947.068280   
                                      Population million               3409.710058   
                        Middle-Income GDP|MER    billion USD_2010/yr  17914.158252   
                                      Population million               2270.461976   
             REF-v2     High-Income   GDP|MER    billion USD_2010/yr  41527.174647   
                                      Population million               1032.375424   
                        Low-Income    GDP|MER    billion USD_2010/yr   8947.068280   
                                      Population million               3409.710058   
                        Middle-Income GDP|MER    billion USD_2010/yr  17914.158252   
                                      Population million               2270.461976   
             TECH-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  41527.174647   
                                      Population million               1032.375424   
                        Low-Income    GDP|MER    billion USD_2010/yr   8947.068280   
                                      Population million               3409.710058   
                        Middle-Income GDP|MER    billion USD_2010/yr  17914.158252   
                                      Population million               2270.461976   

                                                                              2020  \
model        scenario   region        variable   unit                                
GEM-E3_V2023 LIFE-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  42001.582509   
                                      Population million               1051.723709   
                        Low-Income    GDP|MER    billion USD_2010/yr  10261.152858   
                                      Population million               3684.994699   
                        Middle-Income GDP|MER    billion USD_2010/yr  21198.710837   
                                      Population million               2343.921774   
             REF-v2     High-Income   GDP|MER    billion USD_2010/yr  42001.582508   
                                      Population million               1051.723709   
                        Low-Income    GDP|MER    billion USD_2010/yr  10261.152884   
                                      Population million               3684.994699   
                        Middle-Income GDP|MER    billion USD_2010/yr  21198.710853   
                                      Population million               2343.921774   
             TECH-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  42001.582510   
                                      Population million               1051.723709   
                        Low-Income    GDP|MER    billion USD_2010/yr  10261.152940   
                                      Population million               3684.994699   
                        Middle-Income GDP|MER    billion USD_2010/yr  21198.710855   
                                      Population million               2343.921774   

                                                                              2025  \
model        scenario   region        variable   unit                                
GEM-E3_V2023 LIFE-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  47311.769790   
                                      Population million               1066.676412   
                        Low-Income    GDP|MER    billion USD_2010/yr  13270.101082   
                                      Population million               3955.578312   
                        Middle-Income GDP|MER    billion USD_2010/yr  26908.478088   
                                  

# GEM-E3 MER to PPP

In [9]:
df_un_gem.rename(model={'GEM-E3_V2023':'Macro_Economy'}, inplace=True)
df_un_mimosa.rename(model={'MIMOSA':'Macro_Economy'}, inplace=True)
df_un_image.rename(model={'IMAGE 3.3':'Macro_Economy'}, inplace=True)

df_un = pyam.concat([df_un_gem, df_un_mimosa, df_un_image])
df_un = df_un.filter(year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
df_un_to_harmonize = df_un.multiply('GDP|MER', 'GDP|Ratio', 'GDP|PPP|Narrative Effects', ignore_units='billion USD_2010/yr')

df_un_hist = df_un.filter(variable='GDP|PPP|Input')
df_un_hist.rename(variable={'GDP|PPP|Input':'GDP|PPP|Narrative Effects'}, inplace=True)
df_un_to_harmonize.filter(region='Africa*').timeseries()

2020  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  17561.572373   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  17561.572436   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  17561.572573   

                                                                                               2030  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  25612.682806   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  25943.590676   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  25602.083311   

                                                                                               2040  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  36745.165973   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  37548.215430   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  36718.849664   

                                                                                               2050  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  50895.211779   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  51843.008527   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  51018.948032   

                                                                                               2060  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  67631.790112   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  68442.299022   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  67889.959674   

                                                                                               2070  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  77576.519724   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  78074.607651   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  78064.152736   

                                                                                               2080  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  88017.698355   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  87555.626546   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  88653.407175   

                                                                                                2090  \
model         scenario   region         variable                  unit                                 
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  100069.881717   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr   98158.467924   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects b

In [10]:
df_3_groups_gem.rename(model={'GEM-E3_V2023':'Macro_Economy'}, inplace=True)
df_3_goups_mimosa.rename(model={'MIMOSA':'Macro_Economy'}, inplace=True)
df_3_image.rename(model={'IMAGE 3.3':'Macro_Economy'}, inplace=True)

df_3 = pyam.concat([df_3_groups_gem, df_3_goups_mimosa, df_3_image])
df_3 = df_3.filter(year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
df_3_to_harmonize = df_3.multiply('GDP|MER', 'GDP|Ratio', 'GDP|PPP|Narrative Effects', ignore_units='billion USD_2010/yr')
df_3_hist = df_3.filter(variable='GDP|PPP|Input')
df_3_hist.rename(variable={'GDP|PPP|Input':'GDP|PPP|Narrative Effects'}, inplace=True)
df_3_to_harmonize.timeseries()

2020  \
model         scenario   region        variable                  unit                                
Macro_Economy LIFE-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  39300.526929   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  28167.380403   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  45557.294892   
              REF-v2     High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  39300.526928   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  28167.380475   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  45557.294926   
              TECH-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  39300.526930   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  28167.380629   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  45557.294931   

                                                                                              2030  \
model         scenario   region        variable                  unit                                
Macro_Economy LIFE-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  48037.407112   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  42753.038969   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  64205.077193   
              REF-v2     High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  48356.838236   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  43418.455230   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  64873.264391   
              TECH-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  48050.864091   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  42781.188188   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  64252.098987   

                                                                                              2040  \
model         scenario   region        variable                  unit                                
Macro_Economy LIFE-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  56544.660493   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  61436.486618   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  74947.153277   
              REF-v2     High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  57208.903947   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  63077.823430   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  77152.522680   
              TECH-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  56608.457286   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  61476.534061   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  75174.998173   

                                                                                              2050  \
model         scenario   region        variable                  unit                                
Macro_Economy LIFE-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  66870.933605   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  85019.349030   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  84436.730729   
              REF-v2     High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  67858.656195   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  87118.821336   
           

# Harmonize to SSP2 Input

In [11]:
# AIM will be harmonized to IMAGE REF historic values
def harmonize_df(df_gem, df_image_hist):

    # # Create a dataframe that contains the method of harmonization per variable
    # # Only create methods for variables present in AIM
    methods = df_gem.timeseries().reset_index(level='model', drop=True)
    # The offset methods ensure internal regional consistency better than the ratio methods
    methods['method'] = 'constant_offset' # 'reduce_ratio_2080' 'constant_ratio' 'constant_offset' 'reduce_offset_2080'
    methods_series = methods['method']

    hist_to_harmonize = df_image_hist.timeseries()
    df_gem_to_harmonize = df_gem.timeseries()

    # Remove the model level from index and column names should be strings
    df_gem_to_harmonize = df_gem_to_harmonize.reset_index(level='model', drop=True)
    df_gem_to_harmonize.columns = df_gem_to_harmonize.columns.astype(str)
    hist_to_harmonize = hist_to_harmonize.reset_index(level='model', drop=True)
    hist_to_harmonize.columns = hist_to_harmonize.columns.astype(str)

    # Harmonize 
    h = aneris.harmonize.Harmonizer(df_gem_to_harmonize, hist_to_harmonize, config={'harmonize_year': '2020'})
    harmonized = h.harmonize(methods_series) # methods_copy

    # Add all necessary index columns
    new_level = 'Macro_Economy'
    df_index = harmonized.index.to_frame()  # Convert the MultiIndex to a DataFrame
    df_index['model'] = new_level  # Add the new level
    df_index = df_index[['model', 'scenario', 'region', 'variable', 'unit']]
    new_index = pd.MultiIndex.from_frame(df_index)  # Create a new MultiIndex with the added level
    harmonized.index = new_index

    df_harmonized = pyam.IamDataFrame(harmonized)
    return df_harmonized

df_un_ppp_harmonized = harmonize_df(df_un_to_harmonize, df_un_hist)
df_3_ppp_harmonized = harmonize_df(df_3_to_harmonize, df_3_hist)

INFO:root:Harmonizing with constant_offset
INFO:root:Harmonizing with constant_offset


In [12]:
df_un_harmonized = pyam.concat([df_un, df_un_ppp_harmonized])

df_un_harmonized.subtract(1, 
               'GDP|Climate Damage Corrected|Loss|Relative', 
               'GDP|Climate Damage Corrected|Loss|Relative (Inverted)',
               ignore_units='',
               append=True 
              )
df_un_harmonized.multiply('GDP|PPP|Narrative Effects', 
               'GDP|Climate Damage Corrected|Loss|Relative (Inverted)', 
               'GDP|PPP|Narrative and Climate Effects', 
               ignore_units='billion USD_2010/yr', 
               append=True)
df_un_harmonized.filter(region='World*').timeseries()

2020  \
model         scenario   region variable                                           unit                                 
Macro_Economy LIFE-TP-v2 World  GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP           0.000000   
                                GDP|Climate Damage Corrected|Loss|Relative (Inv...                           1.000000   
                                GDP|MER                                            billion USD_2010/yr   80110.288125   
                                GDP|PPP|Input                                      billion USD_2010/yr  102473.678217   
                                GDP|PPP|Narrative Effects                          billion USD_2010/yr  102473.678217   
                                GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  102473.678217   
                                GDP|Ratio                                                                    1.460084   
                                Population                                         million                7795.136232   
              REF-v2     World  GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP           0.000000   
                                GDP|Climate Damage Corrected|Loss|Relative (Inv...                           1.000000   
                                GDP|MER                                            billion USD_2010/yr   80110.288172   
                                GDP|PPP|Input                                      billion USD_2010/yr  102473.678217   
                                GDP|PPP|Narrative Effects                          billion USD_2010/yr  102473.678217   
                                GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  102473.678217   
                                GDP|Ratio                                                                    1.460084   
                                Population                                         million                7795.136232   
              TECH-TP-v2 World  GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP           0.000000   
                                GDP|Climate Damage Corrected|Loss|Relative (Inv...                           1.000000   
                                GDP|MER                                            billion USD_2010/yr   80110.288231   
                                GDP|PPP|Input                                      billion USD_2010/yr  102473.678217   
                                GDP|PPP|Narrative Effects                          billion USD_2010/yr  102473.678217   
                                GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  102473.678217   
                                GDP|Ratio                                                                    1.460084   
                                Population                                         million                7795.136232   

                                                                                                                 2030  \
model         scenario   region variable                                           unit                                 
Macro_Economy LIFE-TP-v2 World  GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP           0.007138   
                                GDP|Climate Damage Corrected|Loss|Relative (Inv...                           0.992862   
                                GDP|MER                                            billion USD_2010/yr  109010.301617   
                                GDP|PPP|Input                                      billion USD_2010/yr  146035.354670   
                                GDP|PPP|Narrative Effects                          billion USD_2010/yr  148724.513896   
                                GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  147662.919481  

In [13]:
df_3_harmonized = pyam.concat([df_3, df_3_ppp_harmonized])

df_3_harmonized.subtract(1, 
               'GDP|Climate Damage Corrected|Loss|Relative', 
               'GDP|Climate Damage Corrected|Loss|Relative (Inverted)',
               ignore_units='',
               append=True 
              )
df_3_harmonized.multiply('GDP|PPP|Narrative Effects', 
               'GDP|Climate Damage Corrected|Loss|Relative (Inverted)', 
               'GDP|PPP|Narrative and Climate Effects', 
               ignore_units='billion USD_2010/yr', 
               append=True)

df_3_harmonized.filter(region='Low-Income').timeseries()

2020  \
model         scenario   region     variable                                           unit                                
Macro_Economy LIFE-TP-v2 Low-Income GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.000000   
                                    GDP|Climate Damage Corrected|Loss|Relative (Inv...                          1.000000   
                                    GDP|MER                                            billion USD_2010/yr  10261.152858   
                                    GDP|PPP|Input                                      billion USD_2010/yr  12910.957393   
                                    GDP|PPP|Narrative Effects                          billion USD_2010/yr  12910.957393   
                                    GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  12910.957393   
                                    GDP|Ratio                                                                   2.745050   
                                    Population                                         million               3684.994699   
              REF-v2     Low-Income GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.000000   
                                    GDP|Climate Damage Corrected|Loss|Relative (Inv...                          1.000000   
                                    GDP|MER                                            billion USD_2010/yr  10261.152884   
                                    GDP|PPP|Input                                      billion USD_2010/yr  12910.957393   
                                    GDP|PPP|Narrative Effects                          billion USD_2010/yr  12910.957393   
                                    GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  12910.957393   
                                    GDP|Ratio                                                                   2.745050   
                                    Population                                         million               3684.994699   
              TECH-TP-v2 Low-Income GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.000000   
                                    GDP|Climate Damage Corrected|Loss|Relative (Inv...                          1.000000   
                                    GDP|MER                                            billion USD_2010/yr  10261.152940   
                                    GDP|PPP|Input                                      billion USD_2010/yr  12910.957393   
                                    GDP|PPP|Narrative Effects                          billion USD_2010/yr  12910.957393   
                                    GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  12910.957393   
                                    GDP|Ratio                                                                   2.745050   
                                    Population                                         million               3684.994699   

                                                                                                                    2030  \
model         scenario   region     variable                                           unit                                
Macro_Economy LIFE-TP-v2 Low-Income GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.015010   
                                    GDP|Climate Damage Corrected|Loss|Relative (Inv...                          0.984990   
                                    GDP|MER                                            billion USD_2010/yr  16823.957896   
                                    GDP|PPP|Input                                      billion USD_2010/yr  22732.435364   
                                    GDP|PPP|Narrative Effects                          billion USD_2010/yr  27496.615959   
                       

In [20]:
df_total = pyam.concat([df_un_harmonized, df_3_harmonized])
df_total.rename(scenario={
    'LIFE-TP-v2':'LIFE-TP',
    'REF-v2':'REF',
    'TECH-TP-v2':'TECH-TP'
}, inplace=True)

df_total.divide('GDP|PPP|Input', "Population", 'GDP|PPP|Input|Per Capita', ignore_units='thousand USD_2010/yr', append=True)
df_total.divide('GDP|PPP|Narrative Effects', "Population", 'GDP|PPP|Narrative Effects|Per Capita', ignore_units='thousand USD_2010/yr', append=True)
df_total.divide('GDP|PPP|Narrative and Climate Effects', "Population", 'GDP|PPP|Narrative and Climate Effects|Per Capita', ignore_units='thousand USD_2010/yr', append=True)

# Get relative to REF narrative and climate effects
tech_ratio = df_total.filter(variable=[
    'GDP|PPP|Narrative and Climate Effects']).divide("TECH-TP", "REF", "TECH-TP", axis='scenario', ignore_units='')
tech_ratio = tech_ratio.rename(variable={
    'GDP|PPP|Narrative and Climate Effects':'GDP|PPP|Relative to REF',
})
life_ratio = df_total.filter(variable=[
    'GDP|PPP|Narrative and Climate Effects']).divide("LIFE-TP", "REF", "LIFE-TP", axis='scenario', ignore_units='')
life_ratio = life_ratio.rename(variable={
    'GDP|PPP|Narrative and Climate Effects':'GDP|PPP|Relative to REF',
})
ref_percentage = df_total.filter(scenario=[
    'REF']).divide("GDP|PPP|Narrative and Climate Effects", "GDP|PPP|Narrative Effects", "GDP|PPP|Climate Effects", ignore_units='')
ref_percentage_capita = df_total.filter(scenario=[
    'REF']).divide("GDP|PPP|Narrative and Climate Effects|Per Capita", "GDP|PPP|Narrative Effects|Per Capita", "GDP|PPP|Climate Effects|Per Capita", ignore_units='')

df_total = pyam.concat([df_total, tech_ratio, life_ratio, ref_percentage, ref_percentage_capita])

df_total = df_total.timeseries().reset_index()
df_total.loc[:, df_total.columns[5:]] = df_total.loc[:, df_total.columns[5:]].applymap(round_to_0_1_percent)


C:\Users\maycocks\AppData\Local\Temp\2\ipykernel_11540\1457075101.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_total.loc[:, df_total.columns[5:]] = df_total.loc[:, df_total.columns[5:]].applymap(round_to_0_1_percent)


In [21]:
df_total_un = df_total[df_total['region'].isin([
    'Africa (UN-R5)', 
    'Asia and the Pacific (UN-R5)',
    'Eastern Europe (UN-R5)',
    'Latin America and Caribbean (UN-R5)',
    'Western Europe and Other States (UN-R5)',
    'World'])]
df_total_un = df_total_un[df_total_un['variable'].isin([
    # 'GDP|PPP|Input',
    'GDP|PPP|Narrative Effects',
    'GDP|Climate Damage Corrected|Loss|Relative (Inverted)',
    'GDP|PPP|Narrative and Climate Effects',
    # 'GDP|PPP|Input|Per Capita',
    'GDP|PPP|Narrative Effects|Per Capita',
    'GDP|PPP|Narrative and Climate Effects|Per Capita',
    "GDP|PPP|Climate Effects",
    "GDP|PPP|Climate Effects|Per Capita",
    "GDP|PPP|Relative to REF",
    ])].reset_index(drop=True)
df_total_un.to_excel('../../../data/TOD/model_results/to_share/GDP_to_share.xlsx', index=False)
# df_total_un.to_excel('../../../data/SOD/model_results/to_share/GDP_to_share_chapter_20.xlsx', index=False)
# df_total_un.to_excel('../../../data/SOD/model_results/to_share/GDP_to_share_chapter_14.xlsx', index=False)
df_total_un

,model,scenario,region,variable,unit,2020,2030,2040,2050,2060,2070,2080,2090,2100
0,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|Climate Damage Corrected|Loss|Relative (In...,,1.00,0.987,0.975,0.975,0.979,0.983,0.986,0.988,0.99
1,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|PPP|Narrative Effects,billion USD_2010/yr,5570.00,13600.000,24800.000,38900.000,55600.000,65600.000,76000.000,88100.000,101000.00
2,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|PPP|Narrative Effects|Per Capita,thousand USD_2010/yr,2.36,4.890,7.660,10.700,13.900,15.100,16.400,18.100,20.10
3,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|PPP|Narrative and Climate Effects,billion USD_2010/yr,5570.00,13400.000,24100.000,37900.000,54500.000,64400.000,74900.000,87000.000,100000.00
4,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|PPP|Narrative and Climate Effects|Per Capita,thousand USD_2010/yr,2.36,4.820,7.470,10.500,13.600,14.800,16.200,17.900,19.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,Macro_Economy,TECH-TP,World,GDP|PPP|Narrative Effects,billion USD_2010/yr,102000.00,149000.000,193000.000,244000.000,304000.000,350000.000,398000.000,449000.000,500000.00
110,Macro_Economy,TECH-TP,World,GDP|PPP|Narrative Effects|Per Capita,thousand USD_2010/yr,13.10,17.400,20.900,25.000,29.900,33.400,37.200,41.300,45.70
111,Macro_Economy,TECH-TP,World,GDP|PPP|Narrative and Climate Effects,billion USD_2010/yr,102000.00,148000.000,190000.000,240000.000,299000.000,344000.000,391000.000,441000.000,492000.00
112,Macro_Economy,TECH-TP,World,GDP|PPP|Narrative and Climate Effects|Per Capita,thousand USD_2010/yr,13.10,17.300,20.600,24.600,29.400,32.800,36.500,40.600,45.00


In [22]:
df_total_income = df_total[df_total['region'].isin([
    'High-Income', 
    'Low-Income',
    'Middle-Income',
    ])]
df_total_income = df_total_income[df_total_income['variable'].isin([
    # 'GDP|PPP|Input',
    'GDP|PPP|Narrative Effects',
    'GDP|Climate Damage Corrected|Loss|Relative (Inverted)',
    'GDP|PPP|Narrative and Climate Effects',
    # 'GDP|PPP|Input|Per Capita',
    'GDP|PPP|Narrative Effects|Per Capita',
    'GDP|PPP|Narrative and Climate Effects|Per Capita',
    "GDP|PPP|Climate Effects",
    "GDP|PPP|Climate Effects|Per Capita",
    "GDP|PPP|Relative to REF",
    ])].reset_index(drop=True)
df_total_income.to_excel('../../../data/TOD/model_results/to_share/GDP_to_share_chapter_19.xlsx', index=False)
df_total_income

,model,scenario,region,variable,unit,2020,2030,2040,2050,2060,2070,2080,2090,2100
0,Macro_Economy,LIFE-TP,High-Income,GDP|Climate Damage Corrected|Loss|Relative (In...,,1.0,0.996,0.993,0.992,0.992,0.992,0.992,0.992,0.992
1,Macro_Economy,LIFE-TP,High-Income,GDP|PPP|Narrative Effects,billion USD_2010/yr,37900.0,46600.000,55100.000,65400.000,77800.000,87300.000,97400.000,108000.000,120000.000
2,Macro_Economy,LIFE-TP,High-Income,GDP|PPP|Narrative Effects|Per Capita,thousand USD_2010/yr,36.0,43.200,50.000,59.100,70.100,78.300,86.800,95.600,105.000
3,Macro_Economy,LIFE-TP,High-Income,GDP|PPP|Narrative and Climate Effects,billion USD_2010/yr,37900.0,46400.000,54700.000,64900.000,77200.000,86600.000,96700.000,107000.000,119000.000
4,Macro_Economy,LIFE-TP,High-Income,GDP|PPP|Narrative and Climate Effects|Per Capita,thousand USD_2010/yr,36.0,43.000,49.700,58.600,69.500,77.700,86.100,94.800,104.000
5,Macro_Economy,LIFE-TP,High-Income,GDP|PPP|Relative to REF,,1.0,0.994,0.992,0.997,1.010,1.020,1.040,1.060,1.080
6,Macro_Economy,LIFE-TP,Low-Income,GDP|Climate Damage Corrected|Loss|Relative (In...,,1.0,0.985,0.973,0.973,0.977,0.980,0.983,0.986,0.988
7,Macro_Economy,LIFE-TP,Low-Income,GDP|PPP|Narrative Effects,billion USD_2010/yr,12900.0,27500.000,46200.000,69800.000,96500.000,116000.000,137000.000,159000.000,182000.000
8,Macro_Economy,LIFE-TP,Low-Income,GDP|PPP|Narrative Effects|Per Capita,thousand USD_2010/yr,3.5,6.510,9.720,13.400,17.300,19.600,22.200,25.200,28.300
9,Macro_Economy,LIFE-TP,Low-Income,GDP|PPP|Narrative and Climate Effects,billion USD_2010/yr,12900.0,27100.000,44900.000,67900.000,94200.000,113000.000,134000.000,156000.000,179000.000
